In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [4]:
filenames = os.listdir('/kaggle/input/dogs-vs-cats/train/train/')

In [5]:
classes = []
for filename in filenames:
    classes.append(filename[:3])
classes[:5]

['cat', 'dog', 'dog', 'dog', 'dog']

In [6]:
df = pd.DataFrame({'FILENAME':filenames, 'CLASS':classes})

In [7]:
df.head()

,FILENAME,CLASS
0,cat.12461.jpg,cat
1,dog.3443.jpg,dog
2,dog.7971.jpg,dog
3,dog.10728.jpg,dog
4,dog.1942.jpg,dog


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
data_gen = ImageDataGenerator(zoom_range=0.2,
                              rotation_range=0.2,
                             horizontal_flip=True,
                             rescale=1/255)

In [10]:
train_data = data_gen.flow_from_dataframe(df, '/kaggle/input/dogs-vs-cats/train/train/',
                                         x_col = 'FILENAME',
                                         y_col = 'CLASS',
                                         target_size=(224,224))

Found 25000 validated image filenames belonging to 2 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

In [12]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D())
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(128,(5,5), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(256,(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [14]:
model.fit(train_data, epochs=5)

Epoch 1/5
782/782 [==============================] - 440s 548ms/step - loss: 0.6453 - accuracy: 0.6128
Epoch 2/5
782/782 [==============================] - 317s 405ms/step - loss: 0.4972 - accuracy: 0.7586
Epoch 3/5
782/782 [==============================] - 309s 395ms/step - loss: 0.3830 - accuracy: 0.8272
Epoch 4/5
782/782 [==============================] - 309s 395ms/step - loss: 0.3056 - accuracy: 0.8686
Epoch 5/5
782/782 [==============================] - 309s 395ms/step - loss: 0.2600 - accuracy: 0.8875


In [19]:
model.save('cat_dog.h5')